In [ ]:
import pandas as pd
import os
from math import sqrt,pow
from GNNfuncs import *

In [ ]:
artists = pd.read_csv(os.path.join('..','data','artists.dat'), delimiter='\t')
tags = pd.read_csv(os.path.join('..','data','tags.dat'), delimiter='\t',encoding='ISO-8859-1')
user_artists = pd.read_csv(os.path.join('..','data','user_artists.dat'), delimiter='\t')
user_friends = pd.read_csv(os.path.join('..','data','user_friends.dat'), delimiter='\t')
user_taggedartists_timestamps = pd.read_csv(os.path.join('..','data','user_taggedartists-timestamps.dat'), delimiter='\t')
user_taggedartists = pd.read_csv(os.path.join('..','data','user_taggedartists.dat'), delimiter='\t')

In [ ]:
for dfs in [artists,tags,user_artists,user_friends,user_taggedartists,user_taggedartists_timestamps]:
    print(get_df_name(dfs, globals()),'\n',dfs.isna().sum(),'\n')

artists 
 id              0
name            0
url             0
pictureURL    444
dtype: int64 

tags 
 tagID       0
tagValue    0
dtype: int64 

user_artists 
 userID      0
artistID    0
weight      0
dtype: int64 

user_friends 
 userID      0
friendID    0
dtype: int64 

user_taggedartists 
 userID      0
artistID    0
tagID       0
day         0
month       0
year        0
dtype: int64 

user_taggedartists_timestamps 
 userID       0
artistID     0
tagID        0
timestamp    0
dtype: int64 



In [6]:
colnames = ['id','name']
artists2 = artists[colnames]
artists2.head()

,id,name
0,1,MALICE MIZER
1,2,Diary of Dreams
2,3,Carpathian Forest
3,4,Moi dix Mois
4,5,Bella Morte


In [41]:
users = user_artists['userID'].unique()
artistcount = [[user,len(get_artists(user))] for user in users]

In [40]:
not50 = []
for i in range(len(artistcount)):
    if artistcount[i][1] != 50:
        not50.append(artistcount[i])
print(len(not50),len(not50)/len(users))
for x in not50:
    print(x)

63 0.033298097251585626
['user35', 15]
['user38', 49]
['user112', 1]
['user188', 2]
['user192', 30]
['user270', 14]
['user344', 35]
['user374', 4]
['user408', 18]
['user486', 32]
['user493', 33]
['user510', 43]
['user526', 49]
['user542', 7]
['user553', 46]
['user558', 12]
['user613', 34]
['user615', 1]
['user701', 11]
['user712', 38]
['user745', 10]
['user768', 44]
['user769', 33]
['user797', 11]
['user819', 41]
['user845', 28]
['user869', 10]
['user942', 34]
['user995', 38]
['user1004', 41]
['user1013', 1]
['user1014', 25]
['user1115', 5]
['user1205', 14]
['user1262', 21]
['user1266', 3]
['user1290', 32]
['user1307', 1]
['user1323', 48]
['user1334', 4]
['user1397', 14]
['user1432', 9]
['user1513', 48]
['user1543', 27]
['user1552', 12]
['user1603', 1]
['user1611', 4]
['user1634', 4]
['user1665', 37]
['user1675', 13]
['user1693', 44]
['user1731', 1]
['user1744', 15]
['user1758', 1]
['user1773', 10]
['user1799', 43]
['user1810', 22]
['user1851', 43]
['user1859', 37]
['user1868', 36]
['u

## Investigating user-user interactions

In [23]:
# Taken from https://dev.to/josethz00/math-for-devs-cosine-similarity-in-python-2524

def cosine_similarity(vector1: list[float], vector2: list[float]) -> float:
    """Returns the cosine of the angle between two vectors."""
    # the cosine similarity between two vectors is the dot product of the two vectors divided by the magnitude of each vector

    dot_product = 0
    magnitude_vector1 = 0
    magnitude_vector2 = 0

    vector1_length = len(vector1)
    vector2_length = len(vector2)

    if vector1_length > vector2_length:
        # fill vector2 with 0s until it is the same length as vector1 (required for dot product)
        vector2 = vector2 + [0] * (vector1_length - vector2_length)
    elif vector2_length > vector1_length:
        # fill vector1 with 0s until it is the same length as vector2 (required for dot product)
        vector1 = vector1 + [0] * (vector2_length - vector1_length)

    # dot product calculation
    for i in range(len(vector1)):
        dot_product += vector1[i] * vector2[i]

    # vector1 magnitude calculation
    for i in range(len(vector1)):
        magnitude_vector1 += pow(vector1[i], 2)

    # vector2 magnitude calculation
    for i in range(len(vector2)):
        magnitude_vector2 += pow(vector2[i], 2)

    # final magnitude calculation
    magnitude = sqrt(magnitude_vector1) * sqrt(magnitude_vector2)

    # return cosine similarity
    return dot_product / magnitude

In [28]:
def friends_tag_similarity(userID):
    friends = get_friends(userID)
    sims = []
    for friend in friends:
        sims.append(cosine_similarity(get_tags(2),get_tags(friend)))
    return sum(sims)/len(sims)

In [35]:
sims2 = []
for userID in user_artists['userID'].unique():
    sims2.append(friends_tag_similarity(userID))
print(sum(sims2)/len(sims2))

0.30130843520311024
